In [2]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import matplotlib.pyplot as plot
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime

In [6]:
file = "./demographic_KY/census_block_demographic_KY.geojson"  
block_pop = gp.read_file(file)
block_pop.head()

,STATEFP10,GEOID,B02001e1,B02001e2,B02001e3,B02001e4,B02001e5,B02001e6,B03002e12,geometry
0,21,15000US210719201002,612,605,0,0,0,0,16,"MULTIPOLYGON (((-82.75596 37.67747, -82.75698 ..."
1,21,15000US210719210003,817,796,0,21,0,0,0,"MULTIPOLYGON (((-82.82312 37.65597, -82.82220 ..."
2,21,15000US210719210001,1101,1101,0,0,0,0,0,"MULTIPOLYGON (((-82.76833 37.70139, -82.76805 ..."
3,21,15000US210719210002,2506,2397,82,0,0,0,28,"MULTIPOLYGON (((-82.73587 37.62262, -82.73606 ..."
4,21,15000US210719210005,536,536,0,0,0,0,0,"MULTIPOLYGON (((-82.81743 37.66274, -82.81814 ..."


In [7]:
#Rename columns
block_pop.rename(columns = {"B02001e6":"hawaiian"}, inplace = True)
block_pop.rename(columns = {"B03002e12":"hispanic_latino"}, inplace = True)
block_pop.rename(columns = {"B02001e5":"asian"}, inplace = True)
block_pop.rename(columns = {"B02001e2":"white"}, inplace = True)
block_pop.rename(columns = {"B02001e3":"black"}, inplace = True)
block_pop.rename(columns = {"B02001e1":"total"}, inplace = True)
block_pop.rename(columns = {"B02001e4":"indian_alaska"}, inplace = True)

#Drop column
#df = df.drop (columns = ['ALAND10'])
init_shape = block_pop.shape
block_pop.head()

,STATEFP10,GEOID,total,white,black,indian_alaska,asian,hawaiian,hispanic_latino,geometry
0,21,15000US210719201002,612,605,0,0,0,0,16,"MULTIPOLYGON (((-82.75596 37.67747, -82.75698 ..."
1,21,15000US210719210003,817,796,0,21,0,0,0,"MULTIPOLYGON (((-82.82312 37.65597, -82.82220 ..."
2,21,15000US210719210001,1101,1101,0,0,0,0,0,"MULTIPOLYGON (((-82.76833 37.70139, -82.76805 ..."
3,21,15000US210719210002,2506,2397,82,0,0,0,28,"MULTIPOLYGON (((-82.73587 37.62262, -82.73606 ..."
4,21,15000US210719210005,536,536,0,0,0,0,0,"MULTIPOLYGON (((-82.81743 37.66274, -82.81814 ..."


In [4]:
file = "./KY/KY_precinct_boundary_neighbors_high_precision.geojson"  
precincts = gp.read_file(file)
precincts.head()

,area,id,type,centerX,centerY,canonical,neighbors,geometry
0,0.002012,21179D105,ORIGINAL,-85.475213,37.8499945,KY Nelson D105,21179D103,"MULTIPOLYGON (((-85.47258 37.82863, -85.47277 ..."
1,0.005111,21149C102,ORIGINAL,-87.10202099999999,37.53273699999974,KY McLean C102,None,"MULTIPOLYGON (((-87.15110 37.49865, -87.15120 ..."
2,0.020440,21147F106,ORIGINAL,-84.65427800000001,36.66191149999971,KY McCreary F106,None,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,0.006972,21147A104,ORIGINAL,-84.4995435,36.62199949999971,KY McCreary A104,None,"MULTIPOLYGON (((-84.60519 36.59878, -84.60507 ..."
4,0.002376,21147F109,ORIGINAL,-84.517808,36.65518849999971,KY McCreary F109,None,"MULTIPOLYGON (((-84.50946 36.62620, -84.50950 ..."


In [9]:
precinct_demographics = precincts[['id', 'geometry']].copy()

#Add type columns
precinct_demographics.insert(1, 'total', 0)
precinct_demographics.insert(2, 'white', 0)
precinct_demographics.insert(3, 'black', 0)
precinct_demographics.insert(4, 'hispanic_latino', 0)
precinct_demographics.insert(5, 'asian', 0)
precinct_demographics.insert(6, 'indian_alaska', 0)
precinct_demographics.insert(7, 'hawaiian', 0)

precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,21179D105,0,0,0,0,0,0,0,"MULTIPOLYGON (((-85.47258 37.82863, -85.47277 ..."
1,21149C102,0,0,0,0,0,0,0,"MULTIPOLYGON (((-87.15110 37.49865, -87.15120 ..."
2,21147F106,0,0,0,0,0,0,0,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,21147A104,0,0,0,0,0,0,0,"MULTIPOLYGON (((-84.60519 36.59878, -84.60507 ..."
4,21147F109,0,0,0,0,0,0,0,"MULTIPOLYGON (((-84.50946 36.62620, -84.50950 ..."


In [10]:
for i_precinct, row_precinct in precincts.iterrows():
    if i_precinct % 1000 == 0:
            print (datetime.datetime.now().strftime("%H:%M:%S"), i_precinct)    
    for i_block, row_block in block_pop.iterrows():
        
        if row_precinct.geometry.contains(row_block.geometry) or row_precinct.geometry.within(row_block.geometry):
            print ("Precinct", row_precinct['id'], 'contains/within block ', i_block)
            precinct_demographics.at[i_precinct, 'total'] += row_block['total']
            precinct_demographics.at[i_precinct, 'white'] += row_block['white']
            precinct_demographics.at[i_precinct, 'black'] += row_block['black']
            precinct_demographics.at[i_precinct, 'hispanic_latino'] += row_block['hispanic_latino']
            precinct_demographics.at[i_precinct, 'asian'] += row_block['asian']
            precinct_demographics.at[i_precinct, 'indian_alaska'] += row_block['indian_alaska']
            precinct_demographics.at[i_precinct, 'hawaiian'] += row_block['hawaiian']
            block_pop = block_pop.drop(i_block)

11:33:57 0
Precinct 21111H151 contains/within block  683
Precinct 21093A102 contains/within block  1811
Precinct 21093A102 contains/within block  1812
Precinct 21093A102 contains/within block  1904
Precinct 21029D113 contains/within block  2355
Precinct 21227A106 contains/within block  1861
11:40:53 1000
Precinct 21115B109 contains/within block  1789
Precinct 21119C109 contains/within block  2656
11:47:48 2000
Precinct 21111J128 contains/within block  487
Precinct 21047G105 contains/within block  2894
Precinct 21047G105 contains/within block  2895
Precinct 21047G105 contains/within block  2896
Precinct 21047G105 contains/within block  2897
Precinct 21047G105 contains/within block  2899
Precinct 21047G105 contains/within block  3168
Precinct 21047G105 contains/within block  3171
Precinct 21047H102 contains/within block  3109
Precinct 21199D114 contains/within block  2500
Precinct 21151D104 contains/within block  2111
Precinct 21151D106 contains/within block  2112
Precinct 21067B184 cont

In [15]:
precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,21179D105,0,0,0,0,0,0,0,"MULTIPOLYGON (((-85.47258 37.82863, -85.47277 ..."
1,21149C102,0,0,0,0,0,0,0,"MULTIPOLYGON (((-87.15110 37.49865, -87.15120 ..."
2,21147F106,0,0,0,0,0,0,0,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,21147A104,0,0,0,0,0,0,0,"MULTIPOLYGON (((-84.60519 36.59878, -84.60507 ..."
4,21147F109,0,0,0,0,0,0,0,"MULTIPOLYGON (((-84.50946 36.62620, -84.50950 ..."


In [16]:
print (init_shape[0] - block_pop.shape[0])

27


In [17]:
for i_precinct, row_precinct in precincts.iterrows():
    if i_precinct % 1000 == 0:
            print (datetime.datetime.now().strftime("%H:%M:%S"), i_precinct)
            
    for i_block, row_block in block_pop.iterrows():
        intersection = row_precinct.geometry.intersection(row_block.geometry)
        if intersection.area > 0:
            ratio = intersection.area / row_precinct.geometry.area
            precinct_demographics.at[i_precinct, 'total'] += int(row_block['total'] * ratio)
            precinct_demographics.at[i_precinct, 'white'] += int(row_block['white'] * ratio)
            precinct_demographics.at[i_precinct, 'black'] += int(row_block['black'] * ratio)
            precinct_demographics.at[i_precinct, 'hispanic_latino'] += int(row_block['hispanic_latino'] * ratio)
            precinct_demographics.at[i_precinct, 'asian'] += int(row_block['asian'] * ratio)
            precinct_demographics.at[i_precinct, 'indian_alaska'] += int(row_block['indian_alaska'] * ratio)
            precinct_demographics.at[i_precinct, 'hawaiian'] += int(row_block['hawaiian'] * ratio)        

12:57:36 0
13:05:59 1000
13:13:48 2000
13:21:37 3000


In [18]:
precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,21179D105,2804,2727,27,77,0,0,0,"MULTIPOLYGON (((-85.47258 37.82863, -85.47277 ..."
1,21149C102,855,845,5,1,0,0,0,"MULTIPOLYGON (((-87.15110 37.49865, -87.15120 ..."
2,21147F106,1130,1065,0,0,0,0,0,"MULTIPOLYGON (((-84.60974 36.72119, -84.60625 ..."
3,21147A104,947,932,10,1,0,0,0,"MULTIPOLYGON (((-84.60519 36.59878, -84.60507 ..."
4,21147F109,976,938,0,9,0,38,0,"MULTIPOLYGON (((-84.50946 36.62620, -84.50950 ..."


In [19]:
precinct_demographics = precinct_demographics.drop(columns = ['geometry'])
precinct_demographics.to_file("./demographic_KY/precinct_demographic_KY.geojson", driver='GeoJSON')